In [42]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 50)
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import precision_recall_cutoff
from cost_function import cost_function, cost_function_cutoff
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import boxcox
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import confusion_matrix, classification_report, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV


# Reading the csv file
train = pd.read_csv('train_dataset.csv')

# Defining the file to be read from s3 bucket
test = pd.read_csv('test_dataset.csv')
test = test.dropna()
test.shape

(498121, 44)

In [43]:
train.shape

(1879, 45)

In [44]:
train.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud,interaction_1,interaction_2,interaction_3,interaction_4,average_seconds_per_item,trustLevel_1,trustLevel_2,trustLevel_3,trustLevel_4,trustLevel_5,trustLevel_6,totalScanTimeInSeconds_0_1,grandTotal_0_1,lineItemVoids_0_1,scansWithoutRegistration_0_1,quantityModifications_0_1,scannedLineItemsPerSecond_0_1,valuePerSecond_0_1,lineItemVoidsPerPosition_0_1,average_seconds_per_item_01,Labels,Labels_0,Labels_1,Labels_2,Labels_3,Labels_4,Labels_5,interaction_5,interaction_6,interaction_7,interaction_8,heredity_1,heredity_2,heredity_3,interaction_9
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379,0,0,0,0,1,19.27,0,0,0,0,1,0,0.575178,0.547174,0.636364,0.0,0.6,0.004045,0.001370,0.021944,0.000131,0,1,0,0,0,0,0,0.000000,0.002427,0.314722,0,0,0,0,0
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143,0,0,0,0,1,3.95,0,0,1,0,0,0,0.057955,0.273637,0.454545,0.2,0.8,0.019364,0.006689,0.032468,0.000027,1,0,1,0,0,0,0,0.090909,0.015491,0.015859,0,0,0,3,0
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769,0,0,0,0,1,24.39,0,0,1,0,0,0,0.827775,0.621811,0.272727,1.0,1.0,0.001204,0.001083,0.020979,0.000165,5,0,0,0,0,0,1,0.272727,0.001204,0.514719,0,0,0,15,0
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862,0,0,0,0,1,19.40,0,0,0,0,0,1,0.978130,0.923462,0.727273,0.4,0.8,0.002347,0.001361,0.025078,0.000132,2,0,0,1,0,0,0,0.290909,0.001877,0.903266,0,0,0,12,0
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111,0,0,0,0,1,5.27,0,0,0,0,1,0,0.234008,0.815608,0.272727,0.7,0.4,0.009337,0.005007,0.010101,0.000036,4,0,0,0,0,1,0,0.190909,0.003735,0.190858,0,0,0,20,0


In [45]:
# Defining input and target variables
X = train[['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9', 'heredity_2', 'interaction_5']]
Y = train['fraud']

# Splitting the data into train, test, and validation
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2)
X_val, X_test, Y_val, Y_test = train_test_split(X_val, Y_val, test_size = 0.2)


# Scaling the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train, columns = X.columns)

X_test = scaler.fit_transform(X_test)
X_test = pd.DataFrame(X_test, columns = X.columns)

X_val = scaler.fit_transform(X_val)
X_val = pd.DataFrame(X_val, columns = X.columns)


X_val

,totalScanTimeInSeconds,interaction_1,interaction_4,heredity_1,interaction_9,heredity_2,interaction_5
0,0.307523,0.0,1.0,0.0,0.0,0.0,0.000000
1,0.785832,0.0,1.0,0.0,0.0,0.0,0.600000
2,0.177375,0.0,1.0,0.0,0.0,0.0,0.381818
3,0.259747,0.0,1.0,0.0,0.0,0.0,0.036364
4,0.049423,0.0,1.0,0.0,0.0,0.0,0.218182
...,...,...,...,...,...,...,...
295,0.515102,0.0,1.0,0.0,0.0,0.0,0.109091
296,0.831411,0.0,1.0,0.0,0.0,0.0,0.381818
297,0.896760,0.0,1.0,0.0,0.0,0.0,0.327273
298,0.706205,0.0,0.0,0.0,0.0,0.0,0.254545


In [46]:
# Defining top 7, 6, and 5 variables
# Train dataset
X_train_7 = X_train
X_train_6 = X_train[['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9', 'heredity_2']]
X_train_5 = X_train[['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9']]

# Test dataset
X_test_7 = X_test
X_test_6 = X_test[['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9', 'heredity_2']]
X_test_5 = X_test[['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9']]

# Validation dataset
X_val_7 = X_val
X_val_6 = X_val[['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9', 'heredity_2']]
X_val_5 = X_val[['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9']]


### Random Forest Model

### Fitting the model with the top 5 features

In [47]:
## Defining the hyper-parameters for RF
RF_param_grid = {'n_estimators': [100, 300, 500],
                 'min_samples_split': [10, 15], 
                 'min_samples_leaf': [5, 7], 
                 'max_depth' : [3, 5, 7]}

# Defining customized scoring function
my_score_function = make_scorer(cost_function, greater_is_better = True, needs_proba = True)

# Performing GridSearch
RF_grid_search = GridSearchCV(RandomForestClassifier(), RF_param_grid, cv = 3, scoring = my_score_function, n_jobs = -1).fit(X_train_5, Y_train)

# Extracting the best model
RF_model = RF_grid_search.best_estimator_
RF_model

RandomForestClassifier(max_depth=5, min_samples_leaf=7, min_samples_split=10,
                       n_estimators=300)

In [48]:
RF_grid_search.cv_results_

{'mean_fit_time': array([0.21325095, 0.63530405, 1.06999365, 0.22029654, 0.63899247,
        1.06639369, 0.21427846, 0.64567002, 1.08287589, 0.23646299,
        0.6424671 , 1.06815886, 0.22102435, 0.66306361, 1.11953751,
        0.2395304 , 0.65272657, 1.08773843, 0.21868054, 0.64757284,
        1.11659122, 0.21862245, 0.64854487, 1.07830588, 0.2191755 ,
        0.65587147, 1.12085438, 0.2191062 , 0.64961171, 1.08339588,
        0.21977409, 0.65488942, 1.10732452, 0.23629864, 0.64765557,
        0.95461027]),
 'std_fit_time': array([0.00103043, 0.00700096, 0.01459259, 0.00497609, 0.00762637,
        0.01184404, 0.00199365, 0.00437959, 0.01984598, 0.0116372 ,
        0.00229495, 0.00354151, 0.00326249, 0.00410284, 0.00656772,
        0.01760351, 0.00416743, 0.01424873, 0.00312314, 0.00637517,
        0.02579039, 0.00201349, 0.00930834, 0.00524464, 0.00328077,
        0.00173246, 0.03855405, 0.00295729, 0.00301594, 0.00094055,
        0.00225479, 0.00396255, 0.0234275 , 0.01489567, 0.003

### Score

In [50]:
score = RF_grid_search.cv_results_
score['mean_test_score'][0]

-80.0

### Fitting the model with the top 6 features

In [51]:
# Defining customized scoring function
my_score_function = make_scorer(cost_function, greater_is_better = True, needs_proba = True)

# Performing GridSearch
RF_grid_search = GridSearchCV(RandomForestClassifier(), RF_param_grid, cv = 3, scoring = my_score_function, n_jobs = -1).fit(X_train_6, Y_train)

# Extracting the best model
RF_model = RF_grid_search.best_estimator_
RF_model

RandomForestClassifier(max_depth=3, min_samples_leaf=7, min_samples_split=15,
                       n_estimators=500)

### Score

In [52]:
score = RF_grid_search.cv_results_
score['mean_test_score'][0]

-80.0

### Fitting the model with the top 7 features

In [53]:
# Defining customized scoring function
my_score_function = make_scorer(cost_function, greater_is_better = True, needs_proba = True)

# Performing GridSearch
RF_grid_search = GridSearchCV(RandomForestClassifier(), RF_param_grid, cv = 3, scoring = my_score_function, n_jobs = -1).fit(X_train_7, Y_train)

# Extracting the best model
RF_model = RF_grid_search.best_estimator_
RF_model

RandomForestClassifier(max_depth=5, min_samples_leaf=7, min_samples_split=15,
                       n_estimators=300)

### Score

In [54]:
score = RF_grid_search.cv_results_
score['mean_test_score'][0]

-83.33333333333333

## From above three scenarios, the best model is the following:
#### RandomForestClassifier(max_depth=5, min_samples_leaf=7, min_samples_split=10, n_estimators=300) with the top 5 features.

### Support Vector Machine

### Fitting the model with the top 5 features

In [55]:
## Defining the hyper-parameters for svm
svm_param_grid = {'kernel': ['rbf', 'poly', 'sigmoid'], 
                  'C': [0.01, 0.1, 1, 10],
                  'gamma': [0.001, 0.01, 0.1, 1]}


svm_grid_search = GridSearchCV(SVR(), svm_param_grid, cv = 3, scoring = 'neg_mean_squared_error', n_jobs = -1).fit(X_train_5, Y_train)

# Extracting the best model
svm_md = svm_grid_search.best_estimator_
svm_md

SVR(C=1, gamma=0.1)

### Score

In [56]:
score = svm_grid_search.cv_results_
score['mean_test_score'][0]

-0.05417500901366402

### Fitting the model with the top 6 features

In [57]:
svm_grid_search = GridSearchCV(SVR(), svm_param_grid, cv = 3, scoring = 'neg_mean_squared_error', n_jobs = -1).fit(X_train_6, Y_train)

# Extracting the best model
svm_md = svm_grid_search.best_estimator_
svm_md

SVR(C=1, gamma=0.1)

### Score

In [58]:
score = svm_grid_search.cv_results_
score['mean_test_score'][0]

-0.054174347266976536

### Fitting the model with the top 7 features

In [17]:
svm_grid_search = GridSearchCV(SVR(), svm_param_grid, cv = 3, scoring = 'neg_mean_squared_error', n_jobs = -1).fit(X_train_7, Y_train)

# Extracting the best model
svm_md = svm_grid_search.best_estimator_
svm_md

SVR(C=10, gamma=0.1)

### Score

In [18]:
score = svm_grid_search.cv_results_
score['mean_test_score'][0]

-0.05097682227630106

## From above three scenarios, the best model is the following:
#### SVR(C=10, gamma=0.1) with the top 7 features.

### AdaBoost

### Fitting the model with the top 5 features

In [59]:
## Defining the hyper-parameters for RF
Ada_param_grid = {'n_estimators': [100, 300, 500],
                 'base_estimator__min_samples_split': [10, 15], 
                 'base_estimator__min_samples_leaf': [5, 7], 
                 'base_estimator__max_depth' : [3, 5, 7],
                 'learning_rate': [0.001, 0.01, 0.1]}

## Running grid search with 3 fold
Ada_grid_search = GridSearchCV(AdaBoostClassifier(base_estimator = DecisionTreeClassifier()), Ada_param_grid, cv = 3, scoring = 'f1', n_jobs = -1).fit(X_train_5, Y_train)

# Extracting the best model
ada_md = Ada_grid_search.best_estimator_
ada_md

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=7,
                                                         min_samples_leaf=5,
                                                         min_samples_split=15),
                   learning_rate=0.001, n_estimators=300)

### Score

In [60]:
score = Ada_grid_search.cv_results_
score['mean_test_score'][0]

0.5385441643972354

### Fitting the model with the top 6 features

In [62]:
## Running grid search with 3 fold
Ada_grid_search = GridSearchCV(AdaBoostClassifier(base_estimator = DecisionTreeClassifier()), Ada_param_grid, cv = 3, scoring = 'f1', n_jobs = -1).fit(X_train_6, Y_train)

# Extracting the best model
ada_md = Ada_grid_search.best_estimator_
ada_md

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=7,
                                                         min_samples_leaf=5,
                                                         min_samples_split=15),
                   learning_rate=0.001, n_estimators=100)

### Score

In [63]:
score = Ada_grid_search.cv_results_
score['mean_test_score'][0]

0.5385441643972354

### Fitting the model with the top 7 features

In [64]:
## Running grid search with 3 fold
Ada_grid_search = GridSearchCV(AdaBoostClassifier(base_estimator = DecisionTreeClassifier()), Ada_param_grid, cv = 3, scoring = 'f1', n_jobs = -1).fit(X_train_7, Y_train)

# Extracting the best model
ada_md = Ada_grid_search.best_estimator_
ada_md

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5,
                                                         min_samples_leaf=5,
                                                         min_samples_split=10),
                   learning_rate=0.001, n_estimators=100)

### Score

In [65]:
score = Ada_grid_search.cv_results_
score['mean_test_score'][0]

0.5210496050361672

## From above three scenarios as well as the previous models, the best model is the following:
#### AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=7, min_samples_leaf=5, min_samples_split=15), learning_rate=0.001, n_estimators=300) with the top 5 features.